In [1]:
import numpy as np
import pandas as pd

from django.conf import settings
from django.db import connections

import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [2]:
sql='''
SELECT
YEAR(DATE_ADD) Y,
QUARTER(DATE_ADD) q,
MONTH(DATE_ADD) m,
DAY(DATE_ADD) d,
SUM((SELECT SUM(product_quantity) FROM ps17_order_detail d WHERE id_order=o.id_order)) products,
SUM((SELECT SUM(product_quantity) FROM ps17_order_detail d WHERE id_order=o.id_order AND reduction_percent>0)) products_discounted,
COUNT(id_order) orders,
ROUND((
SUM((SELECT SUM(original_wholesale_price) FROM ps17_order_detail d WHERE id_order=o.id_order)) 
    ),2) GBP_cost
,
ROUND(SUM((total_products_wt-total_discounts_tax_incl)/conversion_rate) ,2) GBP_products,

ROUND(SUM((total_shipping_tax_incl)/conversion_rate) ,2) GBP_shipping,
ROUND(SUM((total_paid_real)/conversion_rate) ,2) GBP_paid


FROM `ps17_orders` o
WHERE current_state IN
(SELECT id_order_state FROM ps17_order_state WHERE paid=1)

GROUP BY Y,m,d
ORDER BY Y DESC,m DESC,d DESC
'''

sql


'\nSELECT\nYEAR(DATE_ADD) Y,\nQUARTER(DATE_ADD) q,\nMONTH(DATE_ADD) m,\nDAY(DATE_ADD) d,\nSUM((SELECT SUM(product_quantity) FROM ps17_order_detail d WHERE id_order=o.id_order)) products,\nSUM((SELECT SUM(product_quantity) FROM ps17_order_detail d WHERE id_order=o.id_order AND reduction_percent>0)) products_discounted,\nCOUNT(id_order) orders,\nROUND((\nSUM((SELECT SUM(original_wholesale_price) FROM ps17_order_detail d WHERE id_order=o.id_order)) \n    ),2) GBP_cost\n,\nROUND(SUM((total_products_wt-total_discounts_tax_incl)/conversion_rate) ,2) GBP_products,\n\nROUND(SUM((total_shipping_tax_incl)/conversion_rate) ,2) GBP_shipping,\nROUND(SUM((total_paid_real)/conversion_rate) ,2) GBP_paid\n\n\nFROM `ps17_orders` o\nWHERE current_state IN\n(SELECT id_order_state FROM ps17_order_state WHERE paid=1)\n\nGROUP BY Y,m,d\nORDER BY Y DESC,m DESC,d DESC\n'

In [3]:
cols= ['Y','Q','M','D','products','products_discounted','orders','GBP_cost','GBP_products','GBP_shipping','GBP_paid']
cols

['Y',
 'Q',
 'M',
 'D',
 'products',
 'products_discounted',
 'orders',
 'GBP_cost',
 'GBP_products',
 'GBP_shipping',
 'GBP_paid']

In [5]:
def get_data():
    with connections['presta'].cursor() as cursor:
        cursor.execute(sql,)
        rows = cursor.fetchall()

    return rows    

In [6]:
p = pd.DataFrame(get_data(), columns=cols)
p

,Y,Q,M,D,products,products_discounted,orders,GBP_cost,GBP_products,GBP_shipping,GBP_paid
0,2021,4,12,3,12,None,1,21.44,95.35,6.95,102.30
1,2021,4,12,2,6,None,1,8.43,22.73,6.95,29.68
2,2021,4,12,1,17,None,3,19.66,120.29,20.85,141.14
3,2021,4,11,30,6,None,2,15.80,90.39,13.90,104.29
4,2021,4,11,29,31,28,4,59.93,271.11,27.80,298.91
...,...,...,...,...,...,...,...,...,...,...,...
935,2018,4,10,26,3,None,1,0.00,11.65,3.50,15.15
936,2018,4,10,23,5,None,1,5.28,26.34,3.50,29.84
937,2018,4,10,20,4,None,2,5.28,30.14,12.00,42.14
938,2018,4,10,19,12,1,3,9.24,77.58,10.50,119.25


In [28]:
p['GBP_shipping'] = p['GBP_shipping'].astype(float)
p['GBP_products'] = p['GBP_products'].astype(float)
p['GBP_cost'] = p['GBP_cost'].astype(float)

In [29]:
p['VAT8pc'] = round(p['GBP_products']*0.08,2)

In [24]:
p['GrossM'] = p['GBP_products'].astype(float) - p['VAT8pc'] - p['GBP_cost'].astype(float)

In [27]:
p

,Y,Q,M,D,products,products_discounted,orders,GBP_cost,GBP_products,GBP_shipping,GBP_paid,VAT8pc,GrossM
0,2021,4,12,3,12,None,1,21.44,95.35,6.95,102.30,7.63,66.28
1,2021,4,12,2,6,None,1,8.43,22.73,6.95,29.68,1.82,12.48
2,2021,4,12,1,17,None,3,19.66,120.29,20.85,141.14,9.62,91.01
3,2021,4,11,30,6,None,2,15.80,90.39,13.90,104.29,7.23,67.36
4,2021,4,11,29,31,28,4,59.93,271.11,27.80,298.91,21.69,189.49
...,...,...,...,...,...,...,...,...,...,...,...,...,...
935,2018,4,10,26,3,None,1,0.00,11.65,3.50,15.15,0.93,10.72
936,2018,4,10,23,5,None,1,5.28,26.34,3.50,29.84,2.11,18.95
937,2018,4,10,20,4,None,2,5.28,30.14,12.00,42.14,2.41,22.45
938,2018,4,10,19,12,1,3,9.24,77.58,10.50,119.25,6.21,62.13
